### **RQ1. Does support for specific policies (carbon taxes or emissions standards) shift during elections?**
With a focus on climate policy support, we estimate a **reduced Panel VAR(1)** system including two dependent variables: `cc_pol_tax` (support for carbon taxes) and `cc_pol_car` (support for emissions standards). Each equation includes as predictors the **lagged values** of 11 time-varying variables measured in the panel dataset:

- Harm perceptions: `cc4_world`, `cc4_wealthUS`, `cc4_poorUS`, `cc4_comm`, `cc4_famheal`, `cc4_famecon`
- Willingness to pay: `ccSolve`
- Political identity: `pol_party`, `pol_ideology`
- Climate policy support: `cc_pol_tax`, `cc_pol_car`

The following system of two equations is estimated:

$$
\begin{cases}
\text{cc\_pol\_tax}_t = \alpha_1 + \sum_{j=1}^{11} \beta_{1j} \cdot X_{j,t-1} + \epsilon_{1t} \\
\text{cc\_pol\_car}_t = \alpha_2 + \sum_{j=1}^{11} \beta_{2j} \cdot X_{j,t-1} + \epsilon_{2t}
\end{cases}
$$

Where:

- $ X_{j,t-1} $ denotes the value of the $ j $-th predictor at time $ t-1 $
- $ \alpha_i $ is the intercept for equation $ i $
- $ \beta_{ij} $ is the coefficient for the effect of predictor $ j $ on outcome $ i $
- $ \epsilon_{it} $ is the idiosyncratic error term, clustered at the respondent level

The dependent variables are `cc_pol_tax_t` and `cc_pol_car_t`, representing support for carbon taxes and emissions standards at time $ t $, respectively. All variables are treated as **jointly endogenous**, and the model is estimated using **pooled OLS** with **clustered standard errors by respondent** (`PID`), based on 1,722 complete observations from three survey waves.

#### Python Code for Global Panel VAR(1) – RQ1

In [2]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

# Load your dataset
data = pd.read_csv("../data/5-cleaned_data_3waves.csv", low_memory=False)
data['wave'] = pd.to_numeric(data['wave'])

# Define the 11 predictors
predictors = [
    "cc4_world", "cc4_wealthUS", "cc4_poorUS",
    "cc4_comm", "cc4_famheal", "cc4_famecon",
    "ccSolve", "cc_pol_tax", "cc_pol_car",
    "pol_party", "pol_ideology"
]

# Create lagged versions of all predictors
data = data.sort_values(by=['PID', 'wave'])
for var in predictors:
    data[f"{var}_lag"] = data.groupby('PID')[var].shift(1)

# Drop rows with missing lagged values
lagged_vars = [f"{var}_lag" for var in predictors]
panel_var_data = data.dropna(subset=lagged_vars)

# --- Equation 1: cc_pol_tax_t ---
X1 = panel_var_data[lagged_vars]
y1 = panel_var_data["cc_pol_tax"]
X1 = sm.add_constant(X1)
model1 = sm.OLS(y1, X1).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# --- Equation 2: cc_pol_car_t ---
X2 = panel_var_data[lagged_vars]
y2 = panel_var_data["cc_pol_car"]
X2 = sm.add_constant(X2)
model2 = sm.OLS(y2, X2).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# --- Output both results side-by-side ---
print(summary_col([model1, model2],
                  stars=True,
                  model_names=["cc_pol_tax_t", "cc_pol_car_t"],
                  info_dict={"N": lambda x: f"{int(x.nobs)}"}))


                 cc_pol_tax_t cc_pol_car_t
------------------------------------------
const            0.1068       0.6845***   
                 (0.0802)     (0.0920)    
cc4_world_lag    0.1875***    0.1515***   
                 (0.0306)     (0.0296)    
cc4_wealthUS_lag -0.0505**    -0.0520**   
                 (0.0251)     (0.0218)    
cc4_poorUS_lag   0.0614**     0.0975***   
                 (0.0303)     (0.0302)    
cc4_comm_lag     -0.0271      0.0042      
                 (0.0368)     (0.0333)    
cc4_famheal_lag  -0.0043      0.0004      
                 (0.0349)     (0.0303)    
cc4_famecon_lag  0.0253       -0.0201     
                 (0.0257)     (0.0225)    
ccSolve_lag      0.0925***    0.0169      
                 (0.0224)     (0.0188)    
cc_pol_tax_lag   0.3522***    0.1065***   
                 (0.0304)     (0.0269)    
cc_pol_car_lag   0.1376***    0.4294***   
                 (0.0295)     (0.0326)    
pol_party_lag    0.1018***    0.0374**    
          

#### Panel VAR Analysis – RQ1 
**Support for Carbon Taxes (`cc_pol_tax_t`)**
| Predictor              | Coefficient | Significance | Interpretation |
|------------------------|-------------|--------------|----------------|
| `cc_pol_tax_lag`       | **0.352**   | *** p < 0.01 | Strong temporal stability |
| `cc_pol_car_lag`       | **0.138**   | *** p < 0.01 | Support for emissions standards predicts tax support |
| `ccSolve_lag`          | **0.093**   | *** p < 0.01 | Willingness to pay increases tax support |
| `cc4_world_lag`        | **0.188**   | *** p < 0.01 | Global harm perception increases support |
| `cc4_poorUS_lag`       | **0.061**   | ** p < 0.05  | Concern for poor communities increases support |
| `cc4_wealthUS_lag`     | –0.051      | ** p < 0.05  | Concern for wealthy communities reduces support |
| `pol_party_lag`        | **0.102**   | *** p < 0.01 | More left-leaning identification predicts higher support |
| `pol_ideology_lag`     | 0.048       | Not significant | No independent effect when other predictors included |
| Other harm variables   | Not significant | — | No effect from local or personal harm |

**Support for Emissions Standards (`cc_pol_car_t`)**
| Predictor              | Coefficient | Significance | Interpretation |
|------------------------|-------------|--------------|----------------|
| `cc_pol_car_lag`       | **0.429**   | *** p < 0.01 | Very strong temporal stability |
| `cc_pol_tax_lag`       | **0.107**   | *** p < 0.01 | Carbon tax support reinforces emissions standards support |
| `cc4_world_lag`        | **0.152**   | *** p < 0.01 | Global harm perception increases support |
| `cc4_poorUS_lag`       | **0.098**   | *** p < 0.01 | Concern for poor communities increases support |
| `cc4_wealthUS_lag`     | –0.052      | ** p < 0.05  | Concern for wealthy communities reduces support |
| `pol_party_lag`        | **0.037**   | ** p < 0.05  | More left-leaning affiliation predicts higher support |
| `pol_ideology_lag`     | **0.060**   | ** p < 0.05  | Liberal ideology increases support |
| `ccSolve_lag`          | 0.017       | Not significant | No significant influence from WTP |

The results show that **support for both carbon taxes and emissions standards is highly stable over time**, but also shaped by a consistent set of attitudinal factors. Key findings include:
- **Global concern** and **concern for poor communities** significantly increase support for both policies.
- **Concern for wealthy communities** slightly reduces support, possibly reflecting perceived unfairness or skepticism.
- **Willingness to pay** strongly predicts increased support for carbon taxes, but not for emissions standards — suggesting a closer psychological link between financial commitment and market-based policies.
- **Political party affiliation** consistently predicts increased support, with a stronger effect size than ideological self-placement.

> So, the anwer to RQ1 is **No**

---

### **RQ2. ~~How~~ Does willingness to pay for climate solutions vary during elections, and what factors influence it? ~~Are voters more likely to prioritize low-cost climate solutions during election periods?~~**

This question has two parts:
1. **How does WTP evolve across time (especially around elections)?**
2. **What factors (e.g., harm perception, policy support) influence WTP, and vice versa?**

**What We Can Do with Panel VAR**
| Question                                                                                         | Panel VAR                                           |
|--------------------------------------------------------------------------------------------------|-----------------------------------------------------|
| Is WTP influenced by its **own past values**?                                                    | Yes                                                 |
| Is WTP influenced by **past values of other variables** (e.g., harm perception, policy support)? | Yes                                                 |
| Does WTP **predict** support for climate policies or harm perception?                            | Yes                                                 |
| Does the **system of attitudes shift dynamically** over time?                                    | Yes                                                 |

To examine how willingness to pay for climate solutions is shaped by broader climate-related attitudes and political orientations, we estimate a reduced **Panel VAR(1)** model with a single equation:

- `ccSolve_t` (willingness to financially support climate action)

The equation includes as predictors the **lagged values** of ten time-varying variables representing harm perceptions, policy preferences, and political identity:

- **Harm perceptions**:
  - `cc4_world`: perceived harm to the world
  - `cc4_wealthUS`: harm to wealthy communities in the U.S.
  - `cc4_poorUS`: harm to poor communities in the U.S.
  - `cc4_comm`: harm to local communities
  - `cc4_famheal`: harm to family health
  - `cc4_famecon`: harm to family economic well-being

- **Policy support**:
  - `cc_pol_tax`: support for carbon taxes
  - `cc_pol_car`: support for emissions standards

- **Political identity**:
  - `pol_party`: party identification
  - `pol_ideology`: ideological self-placement

$$
\text{ccSolve}_t = \alpha + \beta_1 \cdot \text{cc4\_world}_{t-1}
+ \beta_2 \cdot \text{cc4\_wealthUS}_{t-1}
+ \beta_3 \cdot \text{cc4\_poorUS}_{t-1}
+ \beta_4 \cdot \text{cc4\_comm}_{t-1}
+ \beta_5 \cdot \text{cc4\_famheal}_{t-1}
+ \beta_6 \cdot \text{cc4\_famecon}_{t-1}
+ \beta_7 \cdot \text{ccSolve}_{t-1}
+ \beta_8 \cdot \text{cc\_pol\_tax}_{t-1}
+ \beta_9 \cdot \text{cc\_pol\_car}_{t-1}
+ \beta_{10} \cdot \text{pol\_party}_{t-1}
+ \beta_{11} \cdot \text{pol\_ideology}_{t-1}
+ \epsilon_t
$$

$$
+ \beta_7 \cdot \text{ccSolve}_{t-1}
+ \beta_8 \cdot \text{cc\_pol\_tax}_{t-1}
+ \beta_9 \cdot \text{cc\_pol\_car}_{t-1}
+ \beta_{10} \cdot \text{pol\_party}_{t-1}
+ \beta_{11} \cdot \text{pol\_ideology}_{t-1}
+ \epsilon_t
$$

Where:
- $ \text{ccSolve}_t $ is willingness to pay at time $ t $
- The predictors are all lagged values from $ t - 1 $
- $ \alpha $ is the intercept term
- $ \epsilon_t $ is the idiosyncratic error term clustered at the individual level

To assess how willingness to pay for climate solutions evolves over time and what factors influence it, we will examine below the `ccSolve_t` equation within a global Panel VAR(1) system. This model will included 11 time-varying attitudinal and political variables, each treated as endogenous and lagged by one wave. The model will be estimated using pooled OLS with cluster-robust standard errors at the individual level, based on 1,722 complete observations across three survey waves.

#### Python Code for Global Panel VAR(1) – RQ2

In [7]:
import pandas as pd
import statsmodels.api as sm

# Load and prep data
data = pd.read_csv("../data/5-cleaned_data_3waves.csv", low_memory=False)
data['wave'] = pd.to_numeric(data['wave'])

variables = ["cc4_world", "cc4_wealthUS", "cc4_poorUS", "cc4_comm", "cc4_famheal", "cc4_famecon", "ccSolve", "cc_pol_tax", "cc_pol_car", "pol_party", "pol_ideology"]

# Create lags
data = data.sort_values(by=['PID', 'wave'])
for var in variables:
    data[f'{var}_lag'] = data.groupby('PID')[var].shift(1)

# Drop missing
lagged_vars = [f'{v}_lag' for v in variables]
panel_var_data = data.dropna(subset=lagged_vars)

# Estimate equation: ccSolve_t
X = panel_var_data[lagged_vars]
y = panel_var_data['ccSolve']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Output
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                ccSolve   R-squared:                       0.069
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     11.66
Date:                Mon, 14 Apr 2025   Prob (F-statistic):           1.65e-20
Time:                        23:04:16   Log-Likelihood:                -2353.4
No. Observations:                1722   AIC:                             4731.
Df Residuals:                    1710   BIC:                             4796.
Df Model:                          11                                         
Covariance Type:              cluster                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.9160      0.087  

### Panel VAR Analysis – RQ2 (Willingness to Pay as Outcome)
| Predictor          | Coefficient | Significance    | Interpretation                                       |
|--------------------|-------------|-----------------|------------------------------------------------------|
| `ccSolve_lag`      | 0.054       | * p = 0.052     | WTP is modestly self-reinforcing over time           |
| `cc_pol_tax_lag`   | **0.116**   | *** p < 0.001   | Support for carbon taxes increases future WTP        |
| `cc4_world_lag`    | 0.064       | † p = 0.076     | Global concern may positively influence WTP          |
| `cc4_comm_lag`     | –0.057      | Not significant | No clear effect from local harm perception           |
| `cc4_famheal_lag`  | 0.039       | Not significant | No effect from family health concern                 |
| `cc4_poorUS_lag`   | 0.030       | Not significant | No effect from concern for poor communities          |
| `cc4_wealthUS_lag` | 0.001       | Not significant | No effect from concern for wealthy communities       |
| `cc4_famecon_lag`  | 0.038       | Not significant | No effect from economic concern                      |
| `cc_pol_car_lag`   | –0.035      | Not significant | Support for emissions standards does not predict WTP |
| `pol_party_lag`    | –0.012      | Not significant | No direct effect from party affiliation              |
| `pol_ideology_lag` | 0.057       | † p = 0.094     | Political ideology may predict slightly higher WTP   |

Model fit: $ R^2 = 0.069 $

Willingness to pay for climate solutions is **weakly stable** over time and appears to be shaped primarily by **prior support for carbon taxes**. Individuals who supported carbon pricing in the previous wave were significantly more likely to express willingness to pay in the next. There is also some evidence that global climate concern and political ideology may be associated with increases in WTP, though these effects are marginally significant.

Other harm perception variables, including concern for local or personal impacts, do not significantly influence WTP in this model. Likewise, support for regulatory policies (`cc_pol_car`) and political party identification do not predict willingness to pay when controlling for all other variables.

> So, the anwer to RQ2 is **No**
> 
> These findings suggest that **willingness to pay is primarily shaped by prior policy alignment**, and that shifts in WTP during elections or other periods are likely driven by broader attitudinal shifts rather than direct political or emotional responses.

---

### **RQ3. ~~How~~ Do perceptions of (personal and community-level) climate change impacts influence policy support during elections, and how?**
To examine how perceived climate impacts shape support for climate policies, we will estimate a reduced **Panel VAR(1)** model with two separate equations:
1. `cc_pol_tax_t` (support for carbon taxes)
2. `cc_pol_car_t` (support for emissions standards)

Each equation included as predictors the **lagged values** of six harm perception variables:
- `cc4_world`: perceived harm to the world
- `cc4_wealthUS`: harm to wealthy communities in the U.S.
- `cc4_poorUS`: harm to poor communities in the U.S.
- `cc4_comm`: harm to local communities
- `cc4_famheal`: harm to family health
- `cc4_famecon`: harm to family economic well-being

$
\begin{cases}
\text{cc\_pol\_tax}_t = \alpha_1 + \beta_{11} \cdot \text{cc4\_world}_{t-1} + \beta_{12} \cdot \text{cc4\_wealthUS}_{t-1} + \beta_{13} \cdot \text{cc4\_poorUS}_{t-1} + \beta_{14} \cdot \text{cc4\_comm}_{t-1} + \beta_{15} \cdot \text{cc4\_famheal}_{t-1} + \beta_{16} \cdot \text{cc4\_famecon}_{t-1} + \epsilon_{1t} \\
\\
\text{cc\_pol\_car}_t = \alpha_2 + \beta_{21} \cdot \text{cc4\_world}_{t-1} + \beta_{22} \cdot \text{cc4\_wealthUS}_{t-1} + \beta_{23} \cdot \text{cc4\_poorUS}_{t-1} + \beta_{24} \cdot \text{cc4\_comm}_{t-1} + \beta_{25} \cdot \text{cc4\_famheal}_{t-1} + \beta_{26} \cdot \text{cc4\_famecon}_{t-1} + \epsilon_{2t}
\end{cases}
$

Where:
- $ \alpha_i $ are equation-specific intercepts
- $ \beta_{ij} $ are the estimated coefficients
- $ \epsilon_{it} $ are individual-specific error terms clustered at the respondent level

All models will be estimated using pooled OLS with cluster-robust standard errors by respondent (`PID`), using 1,722 complete observations across three waves. While no election indicator will be included, the dynamic structure captures attitudinal change across the election period (Wave 3).

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

# Load your dataset
data = pd.read_csv("../data/5-cleaned_data_3waves.csv", low_memory=False)
data['wave'] = pd.to_numeric(data['wave'])

# Define harm perception variables
harm_vars = [
    "cc4_world", "cc4_wealthUS", "cc4_poorUS",
    "cc4_comm", "cc4_famheal", "cc4_famecon"
]

# Create lagged variables
data = data.sort_values(by=['PID', 'wave'])
for var in harm_vars:
    data[f'{var}_lag'] = data.groupby('PID')[var].shift(1)

# Drop missing values for lags
lagged_harms = [f'{v}_lag' for v in harm_vars]
panel_var_data = data.dropna(subset=lagged_harms)

# --- Equation 1: cc_pol_tax_t ---
X1 = panel_var_data[lagged_harms]
y1 = panel_var_data['cc_pol_tax']
X1 = sm.add_constant(X1)
model1 = sm.OLS(y1, X1).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# --- Equation 2: cc_pol_car_t ---
X2 = panel_var_data[lagged_harms]
y2 = panel_var_data['cc_pol_car']
X2 = sm.add_constant(X2)
model2 = sm.OLS(y2, X2).fit(cov_type='cluster', cov_kwds={'groups': panel_var_data['PID']})

# Print both results side by side
print(summary_col([model1, model2],
                  stars=True,
                  model_names=['cc_pol_tax_t', 'cc_pol_car_t'],
                  info_dict={'N': lambda x: f"{int(x.nobs)}"}))


                 cc_pol_tax_t cc_pol_car_t
------------------------------------------
const            1.0551***    1.7491***   
                 (0.0888)     (0.0982)    
cc4_world_lag    0.4280***    0.3819***   
                 (0.0387)     (0.0372)    
cc4_wealthUS_lag -0.0504      -0.0637**   
                 (0.0308)     (0.0257)    
cc4_poorUS_lag   0.1811***    0.2238***   
                 (0.0373)     (0.0355)    
cc4_comm_lag     -0.0018      0.0194      
                 (0.0456)     (0.0392)    
cc4_famheal_lag  -0.0041      -0.0020     
                 (0.0437)     (0.0373)    
cc4_famecon_lag  0.0089       -0.0489*    
                 (0.0304)     (0.0269)    
R-squared        0.3194       0.3308      
R-squared Adj.   0.3170       0.3284      
N                1722         1722        
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


#### **Equation 1: Support for Carbon Taxes (`cc_pol_tax_t`)**
| Predictor         | Coefficient | Significance | Interpretation |
|-------------------|-------------|--------------|----------------|
| `cc4_world_lag`   | **0.428**   | *** p < 0.01 | Global harm perception strongly increases support |
| `cc4_poorUS_lag`  | **0.181**   | *** p < 0.01 | Concern for poor communities increases support |
| `cc4_wealthUS_lag`| –0.050      | Not significant | Weak, negative relationship |
| `cc4_comm_lag`    | –0.002      | Not significant | No measurable effect |
| `cc4_famheal_lag` | –0.004      | Not significant | No measurable effect |
| `cc4_famecon_lag` | 0.009       | Not significant | No measurable effect |
| $ R^2 $         | 0.319       | —            | Moderate model fit |

#### **Equation 2: Support for Emissions Standards (`cc_pol_car_t`)**
| Predictor         | Coefficient | Significance | Interpretation |
|-------------------|-------------|--------------|----------------|
| `cc4_world_lag`   | **0.382**   | *** p < 0.01 | Global harm perception increases support |
| `cc4_poorUS_lag`  | **0.224**   | *** p < 0.01 | Concern for poor communities increases support |
| `cc4_wealthUS_lag`| –0.064      | ** p < 0.05  | Concern for wealthy communities reduces support |
| `cc4_famecon_lag` | –0.049      | * p < 0.10   | Concern about family economic harm slightly decreases support |
| `cc4_comm_lag`    | 0.019       | Not significant | No measurable effect |
| `cc4_famheal_lag` | –0.002      | Not significant | No measurable effect |
| $ R^2 $         | 0.331       | —            | Moderate model fit |

The results indicate that **climate harm perceptions do influence policy support**, but only certain forms of harm perception are significant predictors:
- **Global harm perception (`cc4_world`)** increased support for both carbon taxes and emissions standards.
- **Concern for poor communities (`cc4_poorUS`)** also has a strong, positive effect on both types of climate policy support.
- **Concern for wealthy communities (`cc4_wealthUS`)** slightly decreases support for emissions standards, suggesting that **equity-based reasoning** plays a role in public opinion.
- Surprisingly, **personal or community-level harm perceptions**—including concern for one’s own community, family health, or economic well-being—do **not significantly predict** policy support in this model.

Although the model does not include an explicit election indicator, these relationships hold across a time period that includes a federal election (Wave 3). This suggests that changes in support during elections are likely shaped by more **stable perceptions of global and social harm**, rather than by immediate concerns or reactions to election-related events.

> So, the answer for RQ3 is **No**
> 
> Personal and community-level harm perceptions, such as concern for one’s local community, family health, or finances, do not significantly influence support for carbon taxes or emissions standards during the election period. Instead, **support for both policies is more strongly driven by perceptions of global climate harm and concern for poor communities**. This suggests that climate policy support is shaped more by broad and moral concerns than by personal or local risk.

---

### **RQ4. Does political ~~affiliation or~~ ideology moderates the relationship between perceptions of harm and willingness to pay?**
> **RQ4:** Does political affiliation or ideology **moderate** the relationship between perceptions of harm and willingness to pay for climate solutions?

This is a **moderation question**, asking:
- Does the effect of **perceived climate harm** on **willingness to pay (WTP)** vary depending on an individual’s **political ideology or party affiliation**?

**Moderation ≠ What Panel VAR Does Directly**  
Panel VAR models are designed to estimate **dynamic feedback** between variables over time, treating all variables as **jointly endogenous**. They are not ideally suited to test moderation, because they:
- Do not include **interaction terms** (e.g., `cc4_world × pol_ideology`)
- Do not easily accommodate **group-varying effects** within a single estimation

**What We *Can* Do with Panel VAR for RQ4**  
We can estimate **subgrouped Panel VAR(1)** models using:
- `ccSolve` (willingness to pay) as the dependent variable
- All six harm perception variables (`cc4_world`, `cc4_wealthUS`, `cc4_poorUS`, `cc4_comm`, `cc4_famheal`, `cc4_famecon`) as lagged predictors

Then we **run the model separately by political group**, for example:
- **Liberals**
- **Conservatives**

This allows us to ask:

> "Do the effects of different types of perceived harm on WTP vary by political identity?"

**How to Do It: Subgrouped Panel VAR**  
1. **Create ideological groups** using your `pol_ideology` variable (e.g., 1–2 = conservative, 4–5 = liberal)
2. **Run separate Panel VAR(1) models** for each group:
   - Regress `ccSolve_t` on the six lagged harm perception variables and `ccSolve_lag`
3. **Compare the coefficients** for each harm variable across groups:
   - If `cc4_famheal_lag`, `cc4_comm_lag`, etc. are stronger predictors in one group, that suggests **moderation**.

What This Tells Us  
- If harm perceptions **only predict WTP among conservatives**, it suggests WTP is **issue-driven** in that group and more **ideologically anchored** among liberals
- If the effects are similar across groups, **there may be no moderation**

To assess whether political ideology moderates the relationship between climate harm perceptions and willingness to pay (WTP) for climate solutions, I estimated separate **Panel VAR(1)** models for conservative and liberal respondents. Each model regressed `ccSolve_t` (willingness to pay at time $ t $) on:

In [3]:
import pandas as pd
import statsmodels.api as sm

# Define harm perception variables
harm_vars = [
    "cc4_world", "cc4_wealthUS", "cc4_poorUS",
    "cc4_comm", "cc4_famheal", "cc4_famecon"
]

# Create ideology groups
data['ideology_group'] = data['pol_ideology'].apply(
    lambda x: 'liberal' if x >= 4 else ('conservative' if x <= 2 else 'moderate')
)

# Run models for each ideological group
for group in ['conservative', 'liberal']:
    subgroup = data[data['ideology_group'] == group].copy()
    subgroup = subgroup.sort_values(by=['PID', 'wave'])

    # Create lagged variables
    subgroup['ccSolve_lag'] = subgroup.groupby('PID')['ccSolve'].shift(1)
    for var in harm_vars:
        subgroup[f"{var}_lag"] = subgroup.groupby('PID')[var].shift(1)

    # List of lagged predictors
    lagged_vars = [f"{var}_lag" for var in harm_vars] + ['ccSolve_lag']

    # Drop missing values
    subgroup = subgroup.dropna(subset=lagged_vars)

    # Estimate model: ccSolve_t ~ all harm lags + ccSolve_lag
    X = subgroup[lagged_vars]
    y = subgroup['ccSolve']
    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': subgroup['PID']})

    print(f"\nSubgroup: {group.upper()}")
    print(model.summary())


Subgroup: CONSERVATIVE
                            OLS Regression Results                            
Dep. Variable:                ccSolve   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     6.589
Date:                Tue, 15 Apr 2025   Prob (F-statistic):           3.31e-07
Time:                        09:21:15   Log-Likelihood:                -598.01
No. Observations:                 516   AIC:                             1212.
Df Residuals:                     508   BIC:                             1246.
Df Model:                           7                                         
Covariance Type:              cluster                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const           

- The six lagged harm perception variables:
  - `cc4_world`, `cc4_wealthUS`, `cc4_poorUS`, `cc4_comm`, `cc4_famheal`, `cc4_famecon`
- A lagged dependent variable: `ccSolve_lag`

Political ideology was used to define subgroups:
- **Conservatives**: `pol_ideology` ≤ 2
- **Liberals**: `pol_ideology` ≥ 4

Standard errors were clustered by individual (`PID`). This allows us to test whether the relationship between perceived harm and WTP varies across ideological groups.

**Conservative Respondents**  
| Predictor         | Coefficient | Significance | Interpretation |
|-------------------|-------------|--------------|----------------|
| `cc4_world_lag`    | **+0.131**  | ** p < 0.05 | Global harm perception increases WTP |
| All other harm variables | Not significant | — | No measurable effect |
| `ccSolve_lag`      | +0.073      | Not significant | Weak self-reinforcement |
| $ R^2 $          | 0.082       | — | Moderate explanatory power |

→ Among conservatives, **global harm perception significantly increases willingness to pay**, while other forms of perceived harm do not. This suggests that **issue-based concern**, when framed globally, can motivate financial climate commitment in this group.

**Liberal Respondents**  
| Predictor         | Coefficient | Significance | Interpretation |
|-------------------|-------------|--------------|----------------|
| `ccSolve_lag`      | **+0.151**  | ** p < 0.01 | Strong temporal stability in WTP |
| All harm variables | Not significant | — | No predictive effect on WTP |
| $ R^2 $          | 0.029       | — | Lower explanatory power overall |

→ Among liberals, **none of the harm perception variables significantly predict willingness to pay**. Instead, WTP is driven mainly by its **own past values**, indicating that climate commitment in this group is **more stable and ideologically anchored**, rather than responsive to short-term shifts in perceived threat.

These findings suggest that **political ideology moderates the effect of harm perception on willingness to pay**:

- Among **conservatives**, WTP is more responsive to perceived harm — particularly at the global level — indicating that concern about broad climate risks may motivate engagement among right-leaning individuals.
- Among **liberals**, WTP appears to be **less dependent on perceived harm**, and more internally consistent over time, likely reflecting a pre-existing ideological commitment to climate action.

> So, the answer to RQ4 is **Yes**
> 
> This supports the broader theoretical claim that **liberals and conservatives process climate-related risks differently**, with conservatives requiring stronger threat-based cues to express support, and liberals maintaining support based on pre-existing values.

